Davide Sbetti - 14032

---


# DevDataProd - POC

The goal of this POC for the course Development of Data Products is to create a model able to distinguish between different types of text posts. 

In particular, the core idea is to distinguish "clean posts" (regular posts) from the hate speech ones (racist, sexists, homophobe, religion and general hate speech). 

The generated model will then be used on some input posts provided through a GUI. 

For the POC scope, I decided to use an already existing dataset, taken from https://gombru.github.io/2019/10/09/MMHS/

I start by importing the necessary libraries. 

In [ ]:
import pandas as pd
from collections import Counter
import tensorflow_hub as hub
from tensorflow import keras
import tensorflow as tf

I then read the dataset, uploaded on GitHub for simplicity. 

In [ ]:
data = pd.read_json("https://raw.githubusercontent.com/sbettid/DevDataProd/master/MMHS150K_GT.json")

In [ ]:
data

,2005-04-28 09:09:13.714016256,2003-09-08 11:20:48.816660480,2005-02-20 19:22:48.075374593,2005-04-26 23:35:34.635618305,2002-10-22 02:08:00.215592966,2005-01-29 05:56:59.654303744,2005-04-19 14:14:03.568463874,2005-04-27 07:56:57.693966336,2003-02-21 06:38:34.740666370,2005-02-12 03:20:53.910695936,2005-05-21 19:07:28.016556035,2005-04-28 05:39:17.497733122,2005-02-28 17:52:55.861899264,2005-02-07 15:10:47.731273728,2005-05-01 03:29:52.403578884,2005-05-18 05:05:05.716830208,2003-04-16 07:21:28.194969600,2005-04-15 02:05:22.789228549,2005-02-07 17:03:03.066312704,2005-04-15 09:03:35.829471238,2005-01-24 15:20:19.936182273,2005-02-11 03:39:05.122856961,2003-06-09 16:36:48.624058368,2005-05-05 13:37:41.088591879,2003-07-12 21:56:33.044570119,2003-05-25 08:04:56.309178368,2003-02-07 05:34:39.042408453,2005-04-10 15:06:08.954601472,2005-01-08 08:49:27.488548865,2005-03-01 05:01:21.435860992,2005-02-23 13:55:04.600641538,2002-11-11 20:49:06.593603584,2005-02-06 03:49:42.229884933,2005-02-01 12:20:04.810563584,2005-02-20 04:09:38.234503169,2005-05-19 19:50:29.960761354,2005-05-31 09:01:04.130846722,2003-08-16 01:24:14.472830977,2005-02-24 02:03:42.502285313,2005-05-05 06:47:09.476372480,...,2003-07-01 19:43:03.276916737,2005-05-12 01:30:03.905212416,2005-05-19 18:50:09.609572352,2005-01-24 20:01:58.187225089,2005-04-10 06:05:11.161196546,2003-05-20 18:26:07.746727936,2003-02-21 16:30:47.533596672,2005-04-24 04:10:52.889972737,2005-05-14 02:10:05.993803776,2005-02-27 07:14:33.751326726,2005-03-05 13:01:59.260033024,2003-01-11 04:01:32.947935233,2003-08-21 12:19:13.158508544,2003-07-06 17:22:30.372114432,2005-04-11 12:13:27.617699840,2005-04-14 07:43:29.975951360,2005-02-27 09:29:51.843500037,2005-02-23 06:09:39.977379840,2005-05-03 06:37:07.071369217,2003-02-07 06:05:35.005216768,2002-06-25 18:28:22.015246336,2005-02-23 00:30:49.560907776,2005-02-19 12:02:16.818200576,2005-05-13 13:49:00.721590272,2003-09-15 06:55:44.919306246,2003-06-26 02:44:05.215059969,2005-02-05 14:35:18.725541890,2005-01-14 17:09:15.686621185,2005-04-13 19:43:20.923004929,2005-02-02 23:44:53.541232642,2005-05-19 20:33:32.263710720,2003-08-29 12:59:08.748513280,2005-02-03 00:58:27.815477250,2005-01-11 17:45:52.544374786,2005-04-21 06:47:11.071453184,2005-04-22 11:52:14.472048640,2005-03-09 11:36:38.786846720,2005-01-28 19:50:58.540851200,2005-01-09 10:58:29.233188865,2005-04-28 01:58:34.364530691
img_url,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,http://pbs.twimg.com/ext_tw_video_thumb/106301...,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,http://pbs.twimg.com/ext_tw_video_thumb/111401...,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,http://pbs.twimg.com/media/D1zG0qnX4AAw9SC.jpg,http://pbs.twimg.com/media/D3VwYEKW4AYz4vk.jpg,http://pbs.twimg.com/media/D3fQcCCWAAIG8tO.jpg,http://pbs.twimg.com/media/DoN2KFmXcAAIT-Y.jpg,http://pbs.twimg.com/ext_tw_video_thumb/110817...,http://pbs.twimg.com/tweet_video_thumb/D39S8tb...,http://pbs.twimg.com/media/D3gXfziW4AEUvzx.jpg,http://pbs.twimg.com/media/D2YjTKsWsAA1i5W.jpg,http://pbs.twimg.com/tweet_video_thumb/D1-oQf3...,http://pbs.twimg.com/media/D3j8Lz0WsAArzPx.jpg,http://pbs.twimg.com/tweet_video_thumb/D345PrB...,http://pbs.twimg.com/media/DpQL3kGUcAANrXn.jpg,http://pbs.twimg.com/media/D3QOSgnWkAEvioq.jpg,http://pbs.twimg.com/ext_tw_video_thumb/110779...,http://pbs.twimg.com/media/D3QlGJfXsAAQRNJ.jpg,http://pbs.twimg.com/tweet_video_thumb/D1tcqNc...,http://pbs.twimg.com/media/D2C80RkX0AIw0Fy.jpg,http://pbs.twimg.com/media/DqS9h79XQAE4BVI.jpg,http://pbs.twimg.com/ext_tw_video_thumb/111522...,http://pbs.twimg.com/media/Dq7wIAVU0AAn-8q.jpg,http://pbs.twimg.com/tweet_video_thumb/DqAG5Ah...,http://pbs.twimg.com/media/Dn8ml4LX0AAQTll.jpg,http://pbs.twimg.com/tweet_video_thumb/D3Kv_bl...,http://pbs.twimg.com/media/D1ZeCoxWwAAHJyJ.jpg,http://pbs.twimg.com/media/D2ZHxtqUYAE8wkV.jpg,http://pbs.twimg.com/media/D2SNavMX0AEukQt.jpg,http://pbs.twimg.com/tweet_video_thumb/DmRVZ3M...,http://pbs.twimg.com/media/D180hSJXcAALyKw.j

Let me then transpose the data in order to have the different properties as columns

In [ ]:
data_t = data.T

In [ ]:
data_t

,img_url,labels,tweet_url,tweet_text,labels_str
2005-04-28 09:09:13.714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]"
2003-09-08 11:20:48.816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]"
2005-02-20 19:22:48.075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]"
2005-04-26 23:35:34.635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]"
2002-10-22 02:08:00.215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]"
...,...,...,...,...,...
2005-04-22 11:52:14.472048640,http://pbs.twimg.com/tweet_video_thumb/D3ZUXNw...,"[2, 5, 0]",https://twitter.com/user/status/11141707344720...,@svdate @gtconway3d I would just say hes Donny...,"[Sexist, OtherHate, NotHate]"
2005-03-09 11:36:38.786846720,http://pbs.twimg.com/ext_tw_video_thumb/111036...,"[0, 0, 0]",https://twitter.com/user/status/11103681987868...,@Cheftime_Dev congrats my nigga keep on grindi...,"[NotHate, NotHate, NotHate]"
2005-01-28 19:50:58.540851200,http://pbs.twimg.com/media/D1yluGmXgAEKNG5.jpg,"[0, 1, 0]",https://twitter.com/user/status/11069418585408...,My nigga big shitty https://t.co/e0snJGBgH9,"[NotHate, Racist, NotHate]"
2005-01-09 10:58:29.233188865,http://pbs.twimg.com/tweet_video_thumb/D1azqiz...,"[1, 0, 0]",https://twitter.com/user/status/11052683092331...,did she just say “my nigga” to Rich? &amp; she...,"[Racist, NotHate, NotHate]"


## Compress labels

Each label is composed by the judgement of three different judges. In order to obtain a single classification property, I decided to "compress" these judgements in a single one. Following the adopted strategy: 



1.   If at least two judges agree on the same classification I take this as the selected one (majority voting)
2.   If they all provided different classification then I consider if at least two of them were considering the post as a hate speech one (not clean). In this case the post is labelled as "general hate speech"
3.   Otherwise the post is considered clean



In [ ]:
def compress_labels(row):
    
    #get list of labels
    labs_list = row["labels"]
    
    #get the most common label
    most_common = Counter(labs_list).most_common(1)
    
    #set default label to non-hate
    label = 0
    
    #if at least two reviewers agree on a label set it
    if most_common[0][1] > 1:
        label = most_common[0][0]
    elif sum(labs_list) >= 2: #else if they all give different labels but at least two are hate labels
        label = 5 #general other hate category
    
    return label

The function is the applied to the tweets judgement and their text is converted to lowercases for simplicity. 

In [ ]:
data_t['single_label'] = data_t.apply(compress_labels,axis = 1)

In [ ]:
data_t['tweet_text'] = data_t['tweet_text'].str.lower()

In [ ]:
data_t

,img_url,labels,tweet_url,tweet_text,labels_str,single_label
2005-04-28 09:09:13.714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@friskdontmiss nigga https://t.co/casalwepue,"[Religion, Racist, Homophobe]",5
2003-09-08 11:20:48.816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,my horses are retarded https://t.co/hyhqc6d5wn,"[OtherHate, OtherHate, OtherHate]",5
2005-02-20 19:22:48.075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“nigga on ma momma youngboy be spitting real s...,"[NotHate, NotHate, NotHate]",0
2005-04-26 23:35:34.635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,rt xxsugvngxx: i ran into this holy nigga toda...,"[Racist, NotHate, NotHate]",0
2002-10-22 02:08:00.215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“everybody calling you nigger now!” https://t....,"[Racist, NotHate, Racist]",1
...,...,...,...,...,...,...
2005-04-22 11:52:14.472048640,http://pbs.twimg.com/tweet_video_thumb/D3ZUXNw...,"[2, 5, 0]",https://twitter.com/user/status/11141707344720...,@svdate @gtconway3d i would just say hes donny...,"[Sexist, OtherHate, NotHate]",5
2005-03-09 11:36:38.786846720,http://pbs.twimg.com/ext_tw_video_thumb/111036...,"[0, 0, 0]",https://twitter.com/user/status/11103681987868...,@cheftime_dev congrats my nigga keep on grindi...,"[NotHate, NotHate, NotHate]",0
2005-01-28 19:50:58.540851200,http://pbs.twimg.com/media/D1yluGmXgAEKNG5.jpg,"[0, 1, 0]",https://twitter.com/user/status/11069418585408...,my nigga big shitty https://t.co/e0snjgbgh9,"[NotHate, Racist, NotHate]",0
2005-01-09 10:58:29.233188865,http://pbs.twimg.com/tweet_video_thumb/D1azqiz...,"[1, 0, 0]",https://twitter.com/user/status/11052683092331...,did she just say “my nigga” to rich? &amp; she...,"[Racist, NotHate, NotHate]",0


# Building the model

Let's create a one-hot encoded representation of the label

In [ ]:
target = data_t.pop("single_label")

In [ ]:
one_hot_matrix = tf.one_hot( 
    target, 6, on_value = 1.0, off_value = 0.0, axis =-1) 

In [ ]:
one_hot_matrix

<tf.Tensor: shape=(149823, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)>

And let's convert the dataset to a tensorflow one, in order to use it easily with keras. 

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data_t["tweet_text"].values, one_hot_matrix))

### Train / validation / test splits

The dataset is then shuffled and splitted in three parts: 



1. Train: 70%
2.   Validation: 15%
3. Test: 15%



In [ ]:
dataset = dataset.shuffle(len(dataset))

In [ ]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = int(0.15 * len(dataset))

In [ ]:
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

### RNN

We can now build a RNN that starts with an embedding vector that considers a vocabulary size of 7000 words. 

In [ ]:
VOCAB_SIZE=7000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)

In [ ]:
encoder.adapt(train_dataset.map(lambda text, label: text))

The model is then created by the previously defined embedding layer, a LSTM layer composed by 356 units and then a fully connected deep neural network composed by three layers, with 256, 128 and 64 untis respectively. The output is the final classification out of the 6 possible ones. 

In [ ]:
model = keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=256,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    keras.layers.LSTM(256),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(6, activation="softmax")
])

We can then compile the model

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])

and fit it on the training dataset for 25 epochs

In [ ]:
history = model.fit(train_dataset.batch(32), epochs=25, validation_data=val_dataset.batch(32))

Epoch 1/25
3278/3278 [==============================] - 88s 24ms/step - loss: 0.7627 - accuracy: 0.7507 - val_loss: 0.6614 - val_accuracy: 0.7609
Epoch 2/25
3278/3278 [==============================] - 76s 23ms/step - loss: 0.6689 - accuracy: 0.7578 - val_loss: 0.6459 - val_accuracy: 0.7617
Epoch 3/25
3278/3278 [==============================] - 75s 23ms/step - loss: 0.6474 - accuracy: 0.7628 - val_loss: 0.6193 - val_accuracy: 0.7744
Epoch 4/25
3278/3278 [==============================] - 75s 23ms/step - loss: 0.6271 - accuracy: 0.7682 - val_loss: 0.6023 - val_accuracy: 0.7818
Epoch 5/25
3278/3278 [==============================] - 77s 23ms/step - loss: 0.6110 - accuracy: 0.7750 - val_loss: 0.5857 - val_accuracy: 0.7831
Epoch 6/25
3278/3278 [==============================] - 79s 24ms/step - loss: 0.5915 - accuracy: 0.7832 - val_loss: 0.5554 - val_accuracy: 0.7928
Epoch 7/25
3278/3278 [==============================] - 79s 24ms/step - loss: 0.5679 - accuracy: 0.7904 - val_loss: 0.5354 -

We can see how we obtained an accuracy of 93% on the validation set. Evaluating the model on the test set confirmed the quite high accuracy. 

In [ ]:
model.evaluate(test_dataset.batch(32))

703/703 [==============================] - 4s 5ms/step - loss: 0.1794 - accuracy: 0.9370


[0.17937171459197998, 0.9370355606079102]

We can so then save the model for later use. 

In [ ]:
model.save("model")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


downloading it from Google Colab

In [ ]:
from google.colab import files

In [ ]:
!zip -r /content/model.zip /content/model

  adding: content/model/ (stored 0%)
  adding: content/model/saved_model.pb (deflated 90%)
  adding: content/model/assets/ (stored 0%)
  adding: content/model/variables/ (stored 0%)
  adding: content/model/variables/variables.index (deflated 63%)
  adding: content/model/variables/variables.data-00000-of-00001 (deflated 7%)


In [ ]:
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>